In [1]:
from htm.bindings.sdr import SDR
from htm.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
import copy

In [2]:
class ReflexiveMemory:
  def __init__(self):
    self.previous_sdr = None
    self.pairs = {}

  def add(self, sdr):
    current_sdr = '-'.join(map(str, sdr))
    if(self.previous_sdr != None):
      values = self.pairs.get(self.previous_sdr, {})
      pair_count = values.get(current_sdr, 0)
      pair_count = pair_count + 1
      if self.pairs.get(self.previous_sdr, None) is None:
        self.pairs[self.previous_sdr] = { current_sdr: pair_count }
      else:
        self.pairs[self.previous_sdr][current_sdr] = pair_count
    self.previous_sdr = current_sdr

  def find(self, sdr):
    values = self.pairs.get(sdr, {})
    return_value = 0
    return_key = None
    for key, value in values.items():
      if value > return_value:
        return_value = value
        return_key = key
    return return_value, return_key


In [3]:
inputDimensions = (32, 32)
activeDimensions = (64, 64)

config = {
    'sp': {
        'inputDimensions': inputDimensions,
        'columnDimensions': activeDimensions,
        'potentialPct': 0.85,
        'potentialRadius': inputDimensions[0] * inputDimensions[1],
        'globalInhibition': True,
        'localAreaDensity': 0.04395604395604396,
        'synPermInactiveDec': 0.006,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'boostStrength': 3.0,
        'wrapAround': True,
        'seed': 1,
        'learn': False
    },
    'tm': {
        'cellsPerColumn': 13,
        'activationThreshold': 17,
        'initialPermanence': 0.21,
        'minThreshold': 10,
        'maxNewSynapseCount': 32,
        'permanenceIncrement': 0.1,
        'permanenceDecrement': 0.1,
        'predictedSegmentDecrement': 0.0,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'learn': True
    }
}


In [4]:
inputSDR  = SDR(dimensions = inputDimensions)
activeSDR = SDR(dimensions = activeDimensions)

sp = SpatialPooler(
    inputDimensions = config['sp']['inputDimensions'],
    columnDimensions = config['sp']['columnDimensions'],
    potentialPct = config['sp']['potentialPct'],
    potentialRadius = config['sp']['potentialRadius'],
    globalInhibition = config['sp']['globalInhibition'],
    localAreaDensity = config['sp']['localAreaDensity'],
    synPermInactiveDec = config['sp']['synPermInactiveDec'],
    synPermActiveInc = config['sp']['synPermActiveInc'],
    synPermConnected = config['sp']['synPermConnected'],
    boostStrength = config['sp']['boostStrength'],
    wrapAround = config['sp']['wrapAround'],
    seed = config['sp']['seed']
  )

tm = TemporalMemory(
    columnDimensions = config['sp']['columnDimensions'],
    cellsPerColumn = config['tm']['cellsPerColumn'],
    activationThreshold = config['tm']['activationThreshold'],
    initialPermanence = config['tm']['initialPermanence'],
    connectedPermanence = config['sp']['synPermConnected'],
    minThreshold = config['tm']['minThreshold'],
    maxNewSynapseCount = config['tm']['maxNewSynapseCount'],
    permanenceIncrement = config['tm']['permanenceIncrement'],
    permanenceDecrement = config['tm']['permanenceDecrement'],
    predictedSegmentDecrement = config['tm']['predictedSegmentDecrement'],
    maxSegmentsPerCell = config['tm']['maxSegmentsPerCell'],
    maxSynapsesPerSegment = config['tm']['maxSynapsesPerSegment']
  )

rm = ReflexiveMemory()

In [5]:
inputSequence = []
for i in range(10):
    inputSDR.randomize(0.02)
    inputSequence.append(copy.deepcopy(inputSDR.sparse))

inputSequence.append(inputSequence[0])
inputSequence.append(inputSequence[1])
inputSequence.append(inputSequence[2])
inputSequence.append(inputSequence[0])
inputSequence.append(inputSequence[2])

In [6]:
for i in inputSequence:
    inputSDR.sparse = i
    sp.compute(inputSDR, config['sp']['learn'], activeSDR)
    rm.add(copy.deepcopy(activeSDR.sparse))
    tm.compute(activeSDR, learn=config['tm']['learn'])


In [7]:
import hashlib

for key1, value1 in rm.pairs.items():
    for key2, value2 in value1.items():
        result = hashlib.md5(key1.encode())
        print(result.hexdigest(), end=' ')
        result = hashlib.md5(key2.encode())
        print(result.hexdigest(), end=' ')
        print(value2)

ebfc62e8e1ec4dc3f882fade5a8b629d 4823ae145ac1b7c7f7c9c7eb61c4fa94 2
ebfc62e8e1ec4dc3f882fade5a8b629d 50cf4b8ec998d6bbcc7905792d08bdc8 1
4823ae145ac1b7c7f7c9c7eb61c4fa94 50cf4b8ec998d6bbcc7905792d08bdc8 2
50cf4b8ec998d6bbcc7905792d08bdc8 4e5421f56769c5e7f8372a6a2466b541 1
50cf4b8ec998d6bbcc7905792d08bdc8 ebfc62e8e1ec4dc3f882fade5a8b629d 1
4e5421f56769c5e7f8372a6a2466b541 7ce967484df84f2861b2bc55b38fff63 1
7ce967484df84f2861b2bc55b38fff63 1b2114cf29162d044d6680bb7c46e201 1
1b2114cf29162d044d6680bb7c46e201 26ad37443de2dbe0910548d3b0d9a59c 1
26ad37443de2dbe0910548d3b0d9a59c 8a798f6af419c3033e73eddaf6344410 1
8a798f6af419c3033e73eddaf6344410 4006b4669014d87e3212ccedf3ced92b 1
4006b4669014d87e3212ccedf3ced92b 980839decaca03f469f99af53cd58eff 1
980839decaca03f469f99af53cd58eff ebfc62e8e1ec4dc3f882fade5a8b629d 1
